# 在Europe PubMed Central上爬数据

## 背景介绍

Europe PubMed Central的官网在[这里](https://europepmc.org)，维基百科的链接在[这里](https://en.wikipedia.org/wiki/Europe_PubMed_Central)，上面有这个网站很详细的介绍，这里不做重复。简单来说，Europe PMC是大名鼎鼎的PubMed Central的一个镜像网站。这次就是想爬取该网站上的文献资料，主要包括文献标题、摘要、作者、关键字等，不包含全文内容（当然全文内容也是可以获得的）。

这里就有个问题，既然Europe PMC仅仅是PMC的一个镜像网站，内容不如PMC丰富，那为什么不直接爬PMC上的内容呢？原因很简单，因为懒。这两个网站都有提供API服务，但是前者的API调用更加简单，而后者则提供了一整套服务系统（[E-utilities](https://www.ncbi.nlm.nih.gov/books/NBK25501/)，光手册就有150+页）。

显而易见，为了节约时间（不去看那厚厚一沓手册），我选择爬取前者的数据。

## 分析API
Europe PMC提供的接口服务的详细内容可以在[这里](https://europepmc.org/RestfulWebService)找到。

再回到我们的目的：通过特定关键字在Europe PMC上进行检索，获取所有检索结果的信息（主要是文献摘要、标题、作者、关键字等）。结合网站提供的API，选择`/searchPOST`API来完成我们的任务。

`/searchPOST`的使用方法非常简单，无非是通过给出的参数表格，构造一个自定义的request body，然后以表单格式传人URL，通过POST请求返回数据。

一般的参数根据自己的需要取选择就可以了，唯一需要注意的是`cursorMark`这个参数。`cursorMark`的作用是确定检索结果列表的分页。对于第一个请求页，该值可以为空，或者默认值\*，对于其他返回结果，均可以在上一次的返回结果中找到`nextCursorMark`参数，并用来重新构造表单以访问下一页的数据。

In [2]:
import urllib
from bs4 import BeautifulSoup
import codecs
import time

In [ ]:
# 利用Europe PubMed Central提供的API爬取数据
# 不要运行！！！

def get_html(url, headers):
    headers = urllib.parse.urlencode(headers).encode('utf-8')
    response = urllib.request.urlopen(url, headers)
    # print(response)
    html = response.read().decode('utf-8')
    soup = BeautifulSoup(html, 'xml')
    return html, soup


def get_next_cursor_mark(soup):
    next_cursor = soup.find('nextCursorMark').string
    print(next_cursor)
    return next_cursor


def get_abstract_keywords(soup):
    results = soup.find_all('result')
    for result in results:
        title = result.find('title').string
        print("Title: " + title)
        abstract = result.find('abstractText')
        if abstract is not None:
            abstract = abstract.string
            print("Abstract: " + abstract)
        else:
            print('\n')
        key_words = result.find_all('keyword')
        if key_words is not None:
            for kw in key_words:
                if kw.string is not None:
                    print("Keyword: " + kw.string)
                else:
                    print('\n')
        else:
            print('\n')


def save_html(html, filename):
    with codecs.open("./data/" + filename + '.xml', 'w', 'utf-8') as f:
        f.write(html)
    print('文件保存成功')


if __name__ == '__main__':
    # 构造URL和表单数据
    epm_url = "https://www.ebi.ac.uk/europepmc/webservices/rest/searchPOST"
    form_data = {
        'query': 'diabetes',
        'resultType': 'core',
        'synonym': 'true',
        'cursorMark': '*',
        'pageSize': 1000,
        'format': 'xml'
    }
    next_cursor = "AoIIP/GEaSgzODQ2NzQwMQ=="
    form_data['cursorMark'] = next_cursor
    html, soup = get_html(epm_url, form_data)
    count = 79
    save_html(html, 'result' + str(count))
    # get_abstract_keywords(soup)
    count = count + 1
    next_cursor = get_next_cursor_mark(soup)
    while next_cursor is not None:
        form_data['cursorMark'] = next_cursor
        html, soup = get_html(epm_url, form_data)
        # get_abstract_keywords(soup)
        save_html(html, 'result' + str(count))
        count = count + 1
        next_cursor = get_next_cursor_mark(soup)
        time.sleep(1)

## 运行结果

上面的代码运行结果并没有想象中的顺利。在存储了42个文件（即42000条文献）后程序无响应，没有出现报错。终止程序后，人为调整参数`cursorMark`，继续运行，直到下载了78个文件后，程序继续无响应。此时终止实验，不再人为调整，将实验数据（共计78000条文献资料）储存好，等待后续分析。

## 如何改进

首先考虑到添加多线程或者多进程。在学习了[Python中的多线程与多进程](https://github.com/focusxyhoo/my-notebook/blob/master/Python/Python中的多线程与多进程.ipynb)之后，我决定采用多进程的方法。但存在一个问题：在我们这个例子中，对每一页的爬取（第一页除外），都是需要通过上一页返回结果中的`nextCursorMark`参数来构造新的表单。因此我认为很难在程序中加入多进程。

问题暂且先放在这里，后面学习到新的解决办法再来填坑。

## 提取数据
前面的步骤已经帮我们将数据从网站服务器中爬取到本地了。存储的文件格式是`.xml`，已经是格式化的数据了，这样我们不管是提取数据（文献标题、摘要、作者、关键字等）还是进行分析都会非常简单。